In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import DeltaTable
from pyspark.sql.window import Window


In [3]:
source_schema = 'silver_sales'

target_schema = 'gold_sales'

surrogate_Key = 'DimCustomerKey'

key_col = 'CustomerID'

In [4]:
df_src = spark.sql(f"""
SELECT 
	c.CustomerID, c.PersonID, c.AccountNumber,
	p.PersonType, p.Title, p.FirstName, p.MiddleName, p.LastName, p.EmailPromotion,
	ad.AddressLine1, ad.City, ad.PostalCode,
	adt.`Name` AS AddressType,
	st.StateProvinceCode, st.`Name` AS StateProvinceName, st.IsOnlyStateProvinceFlag,
	ter.CountryRegionCode AS CountryCode, ter.`Name` AS CountryName, ter.`Group`,
	pp.PhoneNumber, pnt.`Name` AS PhoneNumberType,
	eml.EmailAddress,
	CURRENT_TIMESTAMP() as StartDate,
    CAST('3000-12-31' AS TIMESTAMP) as EndDate,
    'Y' as IsCurrent
FROM 
{source_schema}.retailcustomer c
LEFT JOIN {source_schema}.Person p
ON c.PersonID = p.BusinessEntityID
LEFT JOIN {source_schema}.BusinessEntityAddress as bea
ON c.PersonID = bea.BusinessEntityID
LEFT JOIN {source_schema}.Address as ad
ON bea.AddressID = ad.AddressID
LEFT JOIN {source_schema}.AddressType as adt
ON bea.AddressTypeID = adt.AddressTypeID
LEFT JOIN {source_schema}.StateProvince st
ON ad.StateProvinceID = st.StateProvinceID
LEFT JOIN {source_schema}.CountryRegion cr
ON st.CountryRegionCode = cr.CountryRegionCode
LEFT JOIN {source_schema}.SalesTerritory ter
ON st.TerritoryID = ter.TerritoryID
LEFT JOIN {source_schema}.PersonPhone pp
ON p.BusinessEntityID = pp.BusinessEntityID
LEFT JOIN {source_schema}.PhoneNumberType pnt
ON pp.PhoneNumberTypeID = pnt.PhoneNumberTypeID
LEFT JOIN {source_schema}.EmailAddress eml
ON p.BusinessEntityID = eml.BusinessEntityID
WHERE StoreID IS NUlL AND adt.`Name` = 'Home'
ORDER BY CustomerID
""")

In [5]:
if spark.catalog.tableExists(F"{target_schema}.DimRetailCustomer"):
    df_trg = spark.sql(f"""
                    SELECT {key_col}, {surrogate_Key}, CreatedAt, UpdatedAt
                    FROM {target_schema}.DimRetailCustomer
                    """)

else:
    df_trg = spark.sql(f"""
                    SELECT 
                    CAST('0' AS INT) AS {key_col}, 
                    CAST('0' AS INT) AS {surrogate_Key}, 
                    CAST('1900-01-01 00:00:00' AS TIMESTAMP) AS CreatedAt,
                    CAST('1900-01-01 00:00:00' AS TIMESTAMP) AS UpdatedAt
                    WHERE 1=0
                    """)

In [6]:
df_src.createOrReplaceTempView("src")
df_trg.createOrReplaceTempView("trg")

df_join = spark.sql(f"""
                SELECT 
                src.*,
                trg.{surrogate_Key},
                trg.CreatedAt,
                trg.UpdatedAt
                FROM
                src LEFT JOIN trg
                ON src.{key_col} = trg.{key_col}
            """)

In [7]:
df_old = df_join.filter(col(f"{surrogate_Key}").isNotNull())

df_new = df_join.filter(col(f"{surrogate_Key}").isNull())

In [8]:
df_old_enr = df_old.withColumn("UpdatedAt", current_timestamp())

In [9]:
if spark.catalog.tableExists(F"{target_schema}.DimRetailCustomer"):
    max_surrogate_key = spark.sql(f"""
                        SELECT MAX({surrogate_Key}) from {target_schema}.DimRetailCustomer
                        """).collect()[0][0]
    w = Window.orderBy(monotonically_increasing_id())

    df_new_enr = df_new.withColumn(f"{surrogate_Key}", row_number().over(w) + lit(max_surrogate_key))\
                        .withColumn("CreatedAt", current_timestamp())\
                        .withColumn("UpdatedAt", current_timestamp())

else:
    max_surrogate_key = 0
    w = Window.orderBy(monotonically_increasing_id())

    df_new_enr = df_new.withColumn(f"{surrogate_Key}", row_number().over(w) + lit(max_surrogate_key))\
                        .withColumn("CreatedAt", current_timestamp())\
                        .withColumn("UpdatedAt", current_timestamp())

max_surrogate_key

In [10]:
df_union = df_old_enr.unionByName(df_new_enr)

df_union.createOrReplaceTempView("df_final")

In [11]:
if spark.catalog.tableExists(f"{target_schema}.DimRetailCustomer"):
    spark.sql(F"""
            MERGE INTO {target_schema}.DimRetailCustomer AS trg
            USING df_final AS src
            ON trg.{surrogate_Key} = src.{surrogate_Key}
            AND trg.IsCurrent = 'Y'

            WHEN MATCHED AND (
                trg.PersonID <> src.PersonID OR
                trg.EmailAddress <> src.EmailAddress OR
                trg.AddressLine1 <> src.AddressLine1 OR
                trg.City <> src.City OR
                trg.PostalCode <> src.PostalCode OR
                trg.PersonType <> src.PersonType OR
                trg.FirstName <> src.FirstName OR
                trg.MiddleName <> src.MiddleName OR
                trg.LastName <> src.LastName OR
                trg.PhoneNumber <> src.PhoneNumber OR
                trg.PhoneNumberType <> src.PhoneNumberType
            )
            THEN UPDATE SET
                trg.IsCurrent = 'N',
                trg.EndDate = Current_Timestamp();
    """)

    spark.sql(f"""
            MERGE INTO {target_schema}.DimRetailCustomer AS trg
            USING df_final AS src
            ON trg.{surrogate_Key} = src.{surrogate_Key}
            AND trg.IsCurrent = 'Y'
            WHEN NOT MATCHED THEN INSERT *

    """)

else:
    df_union.write.format("delta")\
                    .mode("append")\
                    .option("path", "abfss://gold@dlcontoso.dfs.core.windows.net/sales/DimRetailCustomer")\
                    .saveAsTable(f"{target_schema}.DimRetailCustomer")